In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

In [2]:
dtp = pd.read_csv("data/original_data.csv")
dtp.head()

,Unnamed: 0,kartid,rownum,date,time,district,dtp_v,pog,ran,k_ts,...,change_org_motion,s_dtp,coord_w,coord_l,json_str,geo_id,orig_point,g_coder_lat,g_coder_lng,geometry
0,0,168549822,1,2015-05-12,09:20:00,КАШАРСКИЙ,Наезд на пешехода,0,1,1,...,Режим движения не изменялся,820,39.215278,41.385556,"{'KartId': 168549822, 'rowNum': 1, 'date': '12...",NaN,SRID=3857;POINT(4607019.02011651 4752555.37918...,NaN,NaN,POINT (4607019.020116508 4752555.379185724)
1,1,191387240,1,2016-03-09,17:20:00,КАШАРСКИЙ,Опрокидывание,0,1,1,...,Режим движения не изменялся,600,49.157200,41.118300,"{'KartId': 191387240, 'rowNum': 1, 'date': '09...",NaN,SRID=3857;POINT(4577268.21828506 6301577.16111...,NaN,NaN,POINT (4577268.21828506 6301577.161118643)
2,2,210646080,2,2018-08-19,17:40:00,КАШАРСКИЙ,Опрокидывание,0,1,1,...,Режим движения не изменялся,600,48.958862,41.150966,"{'KartId': 210646080, 'rowNum': 2, 'date': '19...",NaN,SRID=3857;POINT(4580904.58077131 6267884.01662...,NaN,NaN,POINT (4580904.580771314 6267884.01662965)
3,3,210542898,3,2018-08-08,14:15:00,КАШАРСКИЙ,Опрокидывание,0,1,2,...,Режим движения не изменялся,30,48.919200,41.145000,"{'KartId': 210542898, 'rowNum': 3, 'date': '08...",NaN,SRID=3857;POINT(4580240.44868924 6261162.42878...,NaN,NaN,POINT (4580240.448689242 6261162.428782783)
4,4,163988162,1,2015-03-07,21:15:00,КАШАРСКИЙ,Наезд на пешехода,1,0,1,...,Движение частично перекрыто,890,48.845556,41.241944,"{'KartId': 163988162, 'rowNum': 1, 'date': '07...",NaN,SRID=3857;POINT(4591032.2054047 6248695.99389284),NaN,NaN,POINT (4591032.205404704 6248695.993892836)


In [3]:
print(dtp.json_str[0])

{'KartId': 168549822, 'rowNum': 1, 'date': '12.05.2015', 'Time': '09:20', 'District': 'КАШАРСКИЙ', 'DTP_V': 'Наезд на пешехода', 'POG': 0, 'RAN': 1, 'K_TS': 1, 'K_UCH': 2, 'infoDtp': {'ndu': ['Не установлены'], 'sdor': ['Перегон (нет объектов на месте ДТП)'], 'ts_info': [{'n_ts': '1', 'ts_s': 'Нет', 't_ts': 'В-класс (малый) до 3,9 м', 'marka_ts': 'KIA', 'm_ts': 'Прочие модели Kia', 'color': 'Иные цвета', 'r_rul': 'Задний', 'g_v': '2013', 'm_pov': 'Передний правый угол | Передний правый бок (передняя дверь пассажира, в случае ТС с правым расположением руля передняя дверь водителя) | Передний левый угол', 't_n': 'Технические неисправности отсутствуют', 'f_sob': 'Частная собственность', 'o_pf': '', 'ts_uch': [{'K_UCH': 'Водитель', 'NPDD': ['Нарушение правил проезда пешеходного перехода'], 'S_T': 'Не пострадал', 'POL': 'Женский', 'V_ST': '3', 'ALCO': '', 'SOP_NPDD': ['Нет нарушений'], 'SAFETY_BELT': 'Да', 'S_SM': 'Нет (не скрывался)', 'N_UCH': '1', 'S_SEAT_GROUP': '', 'INJURED_CARD_ID': ''

In [4]:
dtp.columns

Index(['Unnamed: 0', 'kartid', 'rownum', 'date', 'time', 'district', 'dtp_v',
       'pog', 'ran', 'k_ts', 'k_uch', 'n_p', 'street', 'house', 'dor', 'km',
       'm', 'k_ul', 'dor_k', 'dor_z', 's_pch', 'osv', 'change_org_motion',
       's_dtp', 'coord_w', 'coord_l', 'json_str', 'geo_id', 'orig_point',
       'g_coder_lat', 'g_coder_lng', 'geometry'],
      dtype='object')

In [5]:
# list(dtp.district.value_counts().index)

In [6]:
dtp = dtp[[
    'kartid',
    'date', # дата
    'time', # время
    'district', # район
    'n_p', # населенный пункт
    'dtp_v', # вид дтп
    's_pch', # состояние покрытия дорожного полотна
    'osv', # освещенность
    'change_org_motion', # режим движения (перекрыто/не перекрыто)
    'coord_w', # 
    'coord_l', #
    'json_str'# этот параметр дополнительно будем парсить
]].copy()

In [7]:
def parse_str(x):
    
    x = x.replace("\\xa0", " ")
    
    l = []
    s = 0
    for c in x:
        if c == '"':
            if s == 0:
                s = 1
            else:
                s = 0

            l.append("'")
        elif c == "'":
            if s == 0:
                l.append(c)
        else:
            l.append(c)
            
    return ''.join(l).replace("\'", "\"")


def parse_json(x):
    
    woman_drv = 0 # хотя-бы один из водителей-участников дтп женщина
    alco = 0 # хотя-бы один из водителей-участников дтп в состоянии опьянения
    driver_narush = 0 # хотя-бы один из водителей-участников нарушил пдд
    pogoda = []
    posled = []
    sdor = []
    pdd_narush = []
    
    try:
        _x = json.loads(parse_str(x))
        if 'infoDtp' in _x:
            _info_dtp = _x['infoDtp']
            
            if 's_pog' in _info_dtp:
                pogoda = _info_dtp['s_pog']
                
            if 'sdor' in _info_dtp:
                sdor = _info_dtp['sdor']
                
            if 'ts_info' in _info_dtp:
                _ts_info_arr = _info_dtp['ts_info']
                for _ts_info in _ts_info_arr:
                    if 'ts_uch' in _ts_info:
                        _ts_uch_arr = _ts_info['ts_uch']
                        for _ts_uch in _ts_uch_arr:
                            if ('POL' in _ts_uch) and ('K_UCH' in _ts_uch) and (_ts_uch['K_UCH'] == 'Водитель') and (_ts_uch['POL'] == 'Женский'):
                                woman_drv = 1
                            if ('NPDD' in _ts_uch) and ('K_UCH' in _ts_uch) and (_ts_uch['K_UCH'] == 'Водитель') and ('Нет нарушений' not in _ts_uch['NPDD']):
                                driver_narush = 1
                            if ('ALCO' in _ts_uch) and (_ts_uch['ALCO'] != ''):
                                alco = 1
                            if ('S_T' in _ts_uch):
                                posled.append(_ts_uch['S_T'])
                            if ('NPDD' in _ts_uch) and ('Нет нарушений' not in _ts_uch['NPDD']):
                                pdd_narush.extend(_ts_uch['NPDD'])
                            
            if 'uchInfo' in _info_dtp:
                _uchInfo_arr = _info_dtp['uchInfo']
                for _uchInfo in _uchInfo_arr:
                    if ('S_T' in _uchInfo):
                        posled.append(_uchInfo['S_T'])
#                     if ('NPDD' in _uchInfo) and ('Нет нарушений' not in _uchInfo['NPDD']):
#                         pdd_narush.extend(_uchInfo['NPDD'])
                    
    except Exception:
        print(parse_str(x))
                                
    return pd.Series([woman_drv, alco, driver_narush, '::'.join(pogoda), '::'.join(posled), '::'.join(sdor), '::'.join(pdd_narush)])

In [8]:
# tmp = dtp[dtp['kartid']==171528161].json_str
# print(parse_str(tmp.values[0]))
# print(parse_json(tmp.values[0]))

In [9]:
dtp[['woman_drv', 'alco', 'driver_narush', 'pogoda', 'posled', 'sdor', 'pdd_narush']] = dtp.progress_apply(lambda x: parse_json(x.json_str), axis=1)

100%|██████████| 842261/842261 [08:26<00:00, 1663.55it/s]


In [10]:
# dtp.columns
print('------- N A R U S H E N I E ----------------------')
print(dtp.pdd_narush.value_counts())
print('------- S D O R ----------------------------------')
print(dtp.sdor.value_counts())
print('------- W O M A N --------------------------------')
print(dtp.woman_drv.value_counts())
print('------- A L C O ----------------------------------')
print(dtp.alco.value_counts())
print('------- P O G O D A ------------------------------')
print(dtp.pogoda.value_counts())
print('------- N A R U S H E N I E --- V O D ------------')
print(dtp.driver_narush.value_counts())
print('------- P O S L E D S T V I Y A ------------------')
print(dtp.posled.value_counts())
print('--------------------------------------------------')

------- N A R U S H E N I E ----------------------
Другие нарушения ПДД водителем                                                                                                                                    135292
                                                                                                                                                                   92659
Несоблюдение очередности проезда                                                                                                                                   82253
Нарушение правил проезда пешеходного перехода                                                                                                                      77506
Неправильный выбор дистанции                                                                                                                                       73167
                                                                                                        

In [11]:
dtp = dtp[[
    'date' # дата
    ,'time' # время
    ,'district' # район
    ,'n_p' # населенный пункт
    ,'dtp_v' # вид дтп
    ,'s_pch' # состояние покрытия дорожного полотна
    ,'osv' # освещенность
    ,'change_org_motion' # режим движения (перекрыто/не перекрыто)
#     ,'coord_w' 
#     ,'coord_l'
    ,'woman_drv' # хотя-бы один из водителей-участников дтп женщина
    ,'pogoda' # состояние погоды
    ,'driver_narush'# хотя-бы один из водителей-участников нарушил пдд
    ,'alco' # хотя-бы один из водителей-участников дтп в состоянии опьянения
    ,'posled' # последствия дтп для здоровья участников
    ,'sdor' # объект дорожной сети, на котором произошло дтп
#     ,'pdd_narush' # тип нарушения пдд - очень много всего; много случаев, где нарушений масса; что делать - непонятно
]].copy()

In [ ]:
dtp.to_csv('data/to_encode.csv')